<a href="https://colab.research.google.com/github/thapasamir1704/Image-classification/blob/main/AlexnetCustomization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installing required packages


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import feature
from sklearn import svm
from sklearn import metrics
import sys
%matplotlib inline
import cv2
import seaborn as sns
import tensorflow as tf
%load_ext tensorboard
from numpy import asarray
import shutil
import os
import tensorflow as tf
import zipfile
from tensorflow.keras import layers
from tensorflow.keras import Model
from sklearn.preprocessing import LabelBinarizer
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD
from keras.datasets import cifar10
from tensorflow.keras.optimizers import RMSprop
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import save_model


#Mount and unzip the dataset


In [ ]:
#We mount our google drive to have access to the data files
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Change the directory to the file directory.
%cd #filepath                #Change filepath to the filepath you have.

In [ ]:
!pwd
!unzip dataset.zip


#Data preprocessing and splitting


In [ ]:
#This section aims to shuffle files so that it can produce the same output using a random seed.
import os
import random

# Set the directory path to shuffle the files in
dir_path = #filepath                #Change filepath to the filepath you have.

# Set the random seed for reproducibility
random.seed(10)

# Loop through all subdirectories in the given directory
shuffled_filenames_by_dir = {}
for sub_dir in os.listdir(dir_path):
    sub_dir_path = os.path.join(dir_path, sub_dir)
    # Check if the path is a directory
    if os.path.isdir(sub_dir_path):
        # Get a list of all files in the directory
        files = os.listdir(sub_dir_path)
        # Shuffle the list of files
        random.shuffle(files)
        # Create a list of shuffled filenames for the subdirectory
        shuffled_filenames_by_dir[sub_dir] = files

# Print the shuffled filenames for each subdirectory
for sub_dir, shuffled_filenames in shuffled_filenames_by_dir.items():
    print(f"Shuffled filenames for subdirectory '{sub_dir}':")
    print(shuffled_filenames)

#The list is to check that the list is shuffled in the same order each time. I have run the code multiple times, and can ensure that the list is the same.
#I have also run it in multiple sections instead of running the same code multiple times, and created different lists. The resulting output is the same.


In [ ]:
#We use os to get pathname as well as the folder name which we will be using as labels for our classifiers.
#Change the folder_path to the actual pathname where we unzipped the dataset.
animals = []
import os
folder_path=#filepath                #Change filepath to the filepath you have.
for file in os.listdir(folder_path):
    folder = os.path.join(folder_path, file)
    if os.path.isdir(folder):
        animals.append(os.path.basename(folder))
print (animals)


In [ ]:
#This section counts the number of images for each class so that we can ensure the split into train, valid and test is even for every class.
path = #filepath                #Change filepath to the filepath you have.
def count_files(path):
    file_count = 0
    dir_count = 0
    for root, dirs, files in os.walk(path):
        for name in files:
            file_count += 1
        for name in dirs:
            dir_count += 1
            sub_path = os.path.join(root, name)
            sub_files, sub_dirs = count_files(sub_path)
            file_count += sub_files
            dir_count += sub_dirs
    return file_count, dir_count

if __name__ == '__main__':
    path = #filepath                #Change filepath to the filepath you have.
    img_count = {}
    for root, dirs, files in os.walk(path):
        for name in dirs:
            sub_path = os.path.join(root, name)
            file_count, dir_count = count_files(sub_path)
            sub_dir_name = os.path.basename(sub_path) # get the name of the sub-directory
            img_count[sub_dir_name] = file_count
    print(img_count)
    total_images = sum(img_count.values())
    print("Total Number of Images:", total_images)

In [ ]:
#We are creating three directories where we will be placing all the images according to the split
path = #filepath                #Change filepath to the filepath you have.
%cd #filepath                #Change filepath to the filepath you have.
%mkdir train
%mkdir test
%mkdir valid

In [ ]:
#This section moves 20% of images of each class to a test folder.
path = #filepath                #Change filepath to the filepath you have.
folderCount=0
folderName ='a'
for folder in os.listdir(path):
  for animal in animals:
    if animal in folder:
      %cd /content/gdrive/MyDrive/42028/Assessment2/Image_Classification/dataset_14341022/test
      folderName= animal
      folder_path = os.path.join('/filepath/test', folderName) #Change the path to the test folder you have created.
      if not os.path.exists(folder_path):
        os.mkdir(folderName)
      testPath = '/filepath/test' #Change the path to the test folder you have created.
      newPath = os.path.join(path,animal)
      newTestPath=os.path.join(testPath,animal)
      folderCount=0
      for filename in os.listdir(newPath):
        folderCount+=1
        if (folderCount < 0.2 * (img_count[animal])):
          shutil.move(os.path.join(newPath,filename), newTestPath)
        else:
          continue


In [ ]:
#This section moves all the remaining images(80%) to train folder.
path = #filepath                #Change filepath to the filepath you have
folderName = 'a'
for folder in os.listdir(path):
  for animal in animals:
    if animal in folder:
      %cd /filepath/train #Change this path to the train directory you have created
      newPath = os.path.join(path,animal)
      trainPath = '/filepath/train #Change this path to the train directory you have created.
      shutil.move(newPath,trainPath)

In [ ]:
#This section moves 50% of images from test folder (10% of total images) as well as labels to a valid folder. Now, we have achieved 80-10-10 split.
path = '/filepath/test' #Change this to the test folder you have created.
folderCount=0
folderName ='a'
for folder in os.listdir(path):
  for animal in animals:
    if animal in folder:
      %cd /filepath/valid #Change this to the valid filepath you have
      folderName= animal
      folder_path = os.path.join('/filepath/valid', folderName) ##Change this to the valid filepath you have
      if not os.path.exists(folder_path):
        os.mkdir(folderName)
      validPath = '/filepath/valid' #Change this to the valid filepath you have
      newPath = os.path.join(path,animal)
      newValidPath=os.path.join(validPath,animal)
      folderCount=0
      for filename in os.listdir(newPath):
        folderCount+=1
        if (folderCount < 0.1 * (img_count[animal])):
          shutil.move(os.path.join(newPath,filename), newValidPath)
        else:
          continue

In [ ]:
#This section will display three random images with their labels from the train directory.
import os
import random
import matplotlib.pyplot as plt

def display_random_images_from_directory(directory, num_images=3):
    image_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(('.jpg', '.jpeg', '.png')):
                image_files.append(os.path.join(root, file))

    random_images = random.sample(image_files, min(num_images, len(image_files)))

    for image_path in random_images:
        image = plt.imread(image_path)
        label = os.path.basename(os.path.dirname(image_path))

        plt.imshow(image)
        plt.title(label)
        plt.axis('off')
        plt.show()

if __name__ == '__main__':
    directory_path = '/filepath/train'  # Replace with the path to the directory containing the images
    display_random_images_from_directory(directory_path)


#Creating Base Alex-net architecture

In [ ]:
model_base = tf.keras.models.Sequential([
    #Conv_1          #original model was built for input shape of 224X224
    tf.keras.layers.Conv2D(96, (11,11),strides=4, padding='valid', activation='relu', input_shape=(224, 224, 3)),
    # Pooling_1
    tf.keras.layers.MaxPooling2D((2, 2), strides=(2,2),padding='valid'),
    # Batch Normalisation_1
    tf.keras.layers.BatchNormalization(),
    # Conv_2
    tf.keras.layers.Conv2D(256, (11,11),strides=1, padding='valid', activation='relu'),
    # Pooling_2
    tf.keras.layers.MaxPooling2D((2, 2), strides=(2,2),padding='valid'),
    #Batch Normalisation_2
    tf.keras.layers.BatchNormalization(),
    # Conv_3
    tf.keras.layers.Conv2D(384, (3,3),strides=1, padding='valid', activation='relu'),
    # Batch Normalisation_3
    tf.keras.layers.BatchNormalization(),
    # Conv_4
    tf.keras.layers.Conv2D(384, (3,3),strides=1, padding='valid', activation='relu'),
    # Batch Normalisation_4
    tf.keras.layers.BatchNormalization(),
    #conv_5
    tf.keras.layers.Conv2D(256, (3,3),strides=1, padding='valid', activation='relu'),
    #pooling_3
    tf.keras.layers.MaxPooling2D((2, 2), strides=(2,2),padding='valid'),
    #Batch Normalization_5
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Flatten(),
    #Dense layer_1
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    #Dense layer_2
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    #Dense layer_3
    tf.keras.layers.Dense(1000, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(20, activation='softmax')
    ])

In [ ]:
# Perform image augmentation and create data generators
train_dir = '/filepath/train' #Change this path to the train folder you have.
valid_dir = '/filepath/valid' #Change this path to the validation folder you have.
test_dir =  '/filepath/test' #Change this path to the test folder you have.
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest',
      )
valid_datagen = ImageDataGenerator (rescale = 1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
batch_s =32



# Flow training images in batches of 32 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
  train_dir,
          #Write code here#,  # This is the source directory for training images
  target_size = (224, 224), #Write code here#),  ]
  batch_size=batch_s,
          # Since we use binary_crossentropy loss, we need binary labels
  class_mode='categorical')

validation_generator = valid_datagen.flow_from_directory(
  valid_dir,
          #Write code here#,  # This is the source directory for training images
  target_size = (224, 224), #Write code here#),
  batch_size=batch_s,
          # Since we use binary_crossentropy loss, we need binary labels
  class_mode='categorical')

# Flow validation images in batches of 20 using test_datagen generator
test_generator = test_datagen.flow_from_directory(
  test_dir,
        target_size=(224,224), #Write code here
        batch_size=batch_s,
        class_mode= 'categorical')


In [ ]:
#Compiling the model
model_base.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(learning_rate=1e-4),
              metrics=['acc'])

In [ ]:
#Display Model Summary
from keras.utils import plot_model
from keras.utils.vis_utils import model_to_dot
from IPython.display import HTML, display, clear_output, SVG
model_base.summary()

# Save model architecture as an image file
plot_model(model_base, to_file='model.png', show_layer_names=True, show_shapes=True, rankdir='LR')

# Display model architecture inline in Jupyter Notebook
SVG(model_to_dot(model_base, show_layer_names=True, show_shapes=True, rankdir='LR').create(prog='dot', format='svg'))

In [ ]:
# Train the model
history = model_base.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.n//batch_s,
      epochs=100,
      validation_data=validation_generator,
      validation_steps=validation_generator.n//batch_s,
      verbose=2)

In [ ]:
#Evaluating it on test data
model_base.evaluate_generator(test_generator)




In [ ]:
#Evaluation on validation data
model_base.evaluate_generator(validation_generator)


In [ ]:
#Save the created model
#Change the name to suit your need
save_model(model_base, 'model.h5')

In [ ]:
#Plot training and validation accuracy and loss
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
#This section aims to provide examples of images with their predicted labels and original labels.

# Make predictions on the test data using the trained model
predictions = model_base.predict(test_generator)

# Get the class labels corresponding to the predicted probabilities
predicted_labels = np.argmax(predictions, axis=1)

# Get the original class labels from the test generator
true_labels = test_generator.classes

# Get the class names from the test generator
class_names = list(test_generator.class_indices.keys())

# Get a batch of test images and their corresponding labels
images, _ = test_generator.next()

# Plot the images with their original and predicted labels
plt.figure(figsize=(10, 6))
for i in range(4):
    plt.subplot(2, 2, i + 1)
    plt.imshow(images[i])
    plt.title(f"True: {class_names[true_labels[i]]}\nPredicted: {class_names[predicted_labels[i]]}")
    plt.axis('off')
plt.tight_layout()
plt.show()


#FINAL CUSTOMIZED MODEL

In [ ]:

#In this model, 2 convolution layers (layer 4 and 5) are removed. The convolution layer 3 is resized to 256 nodes from previous 384 nodes.
#Regularizer is used in the last 2 convolution layers (except the input layer).
regularizer = tf.keras.regularizers.l2(0.1)
model_custom = tf.keras.models.Sequential([
    # Conv_1
    tf.keras.layers.Conv2D(96, (11, 11), strides=4, padding='valid', activation='relu', input_shape=(224, 224, 3)),
    # Pooling_1
    tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='valid'),
    # Batch Normalization_1
    tf.keras.layers.BatchNormalization(),
    # Conv_2
    tf.keras.layers.Conv2D(256, (5, 5), strides=1, padding='same', activation='relu', kernel_regularizer=regularizer),
    # Pooling_2
    tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='valid'),
    # Batch Normalization_2
    tf.keras.layers.BatchNormalization(),
    # Conv_3
    tf.keras.layers.Conv2D(256, (3, 3), strides=1, padding='same', activation='relu', kernel_regularizer=regularizer),
    tf.keras.layers.BatchNormalization(),
    # Removed Conv_4
    #tf.keras.layers.Conv2D(256, (3,3),strides=1, padding='valid', activation='relu'),
     #Batch Normalisation_4
    #tf.keras.layers.BatchNormalization(),
    #  Removed Conv_5
   # tf.keras.layers.Conv2D(256, (3, 3), strides=1, padding='same', activation='relu', kernel_regularizer=regularizer),
    #tf.keras.layers.BatchNormalization(),
    # Pooling_3
    #tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='valid'),
    # Batch Normalization_3
    #tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Flatten(),
    # Dense layer_1
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    # Batch Normalization_4
    tf.keras.layers.BatchNormalization(),
    # Dense layer_2
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    # Batch Normalization_5
    tf.keras.layers.BatchNormalization(),
    #Dense layer 3
    tf.keras.layers.Dense(1000, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    # Batch Normalization_5
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(20, activation='softmax')
])


In [ ]:
# Perform image augmentation and create data generators
train_dir = '/filepath/train' #Change this path to the train folder you have.
valid_dir = '/filepath/valid' #Change this path to the validation folder you have.
test_dir =  '/filepath/test' #Change this path to the test folder you have.
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest',
      )
valid_datagen = ImageDataGenerator (rescale = 1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
batch_s =32



# Flow training images in batches of 32 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
  train_dir,
          #Write code here#,  # This is the source directory for training images
  target_size = (224, 224), #Write code here#),  ]
  batch_size=batch_s,
          # Since we use binary_crossentropy loss, we need binary labels
  class_mode='categorical')

validation_generator = valid_datagen.flow_from_directory(
  valid_dir,
          #Write code here#,  # This is the source directory for training images
  target_size = (224, 224), #Write code here#),
  batch_size=batch_s,
          # Since we use binary_crossentropy loss, we need binary labels
  class_mode='categorical')

# Flow validation images in batches of 20 using test_datagen generator
test_generator = test_datagen.flow_from_directory(
  test_dir,
        target_size=(224,224), #Write code here
        batch_size=batch_s,
        class_mode= 'categorical')


In [ ]:
#This section complies the customized model
model_custom.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(learning_rate=1e-4),
              metrics=['acc'])

In [ ]:
#Display Model Summary
from keras.utils import plot_model
from keras.utils.vis_utils import model_to_dot
from IPython.display import HTML, display, clear_output, SVG
model_custom.summary()

# Save model architecture as an image file
plot_model(model_custom, to_file='model.png', show_layer_names=True, show_shapes=True, rankdir='LR')

# Display model architecture inline in Jupyter Notebook
SVG(model_to_dot(model_custom, show_layer_names=True, show_shapes=True, rankdir='TB').create(prog='dot', format='svg'))

In [ ]:
# Train the model
history_custom = model_custom.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.n//batch_s,
      epochs=100,
      validation_data=validation_generator,
      validation_steps=validation_generator.n//batch_s,
      verbose=2)

In [ ]:
#I have used regularization on three dense layers. The train arraucy is 0.8424 and test accuracy is 0.4896. The model is severely overfitting and performing worse than before.
#I had removed one convolution layer. Now, I will remove one dense layer to  see if it improves.
#After I did regularization on two dense layers, the train accuracy rose up to 0.8757 with a loss of 1.7030. However, the validation accuracy was 0.6840 with a loss of 2.8073.  .
#The test accuracy was around 0.657 with a loss of 2.91. It performed better than the previous regularization model, but the model is still overfitting and the accuracy is
#the same as baseline model.
#I will now try to use regularization on convolution layer to see if it makes a difference. I have added regularization to two layers which has the highest number of nodes to see if
#it makes a difference.
#I noticed that the model I made after the base model performed better, so I am customizing it further before doing regularization.

#Then, I removed convolution 4 and last dense layer, and applied regularization with value 0.05 on last 2 convolution layers (3 and 5). The train accuracy was 0.8129 with a loss of 0.8480
#The validation acc was 0.6215 with a alos of 2.0903. The test acc was 0.567 with a loss of 2.49. The model is still overfitting. Now I am gonna experiment value of 0.1
#wHEN i USED REGULARIZATION value of 01, the train accuracy was 0.7847 with a loss of 0.9571 and validation accuracy was 0.6562 with a loss of 1.6501. The test accuracy was 0.6168 with a loss of 1.747. The model is still overfitting.

#Now, I will put back the dense layer I removed, remove the last 2 convolution layers and put the regularizer on last two convolution layers.
#I removed the last 2 convolution layers, and applied regularization on the other  two convolution layers. The initial train accuracy is 0..7388 with a loss of 1.0906, the validation accuracy is 0.6424 with a loss of 1.6521.
#The test accuracy was 0.61999 with a loss of 1.6966
#Now, I am going to change the last convolution layer to be with 256 nodes instead of previous 358.
#The test accuracy is the same 0.6199 with a loss of 1.6968.

In [ ]:
#Performance on test data
model_custom.evaluate_generator(test_generator)

In [ ]:
#Performance on train data
model_custom.evaluate_generator(train_generator)

In [ ]:
#Performance on validation data
model_custom.evaluate_generator(validation_generator)

In [ ]:
#Plots training and validation loss and acccuracy for the customized model
import matplotlib.pyplot as plt
acc = history_custom.history['acc']
val_acc =  history_custom.history['val_acc']
loss =  history_custom.history['loss']
val_loss =  history_custom.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
#This section aims to provide examples of images with their predicted labels and original labels.

# Make predictions on the test data using the trained model
predictions = model_custom.predict(test_generator)

# Get the class labels corresponding to the predicted probabilities
predicted_labels = np.argmax(predictions, axis=1)

# Get the original class labels from the test generator
true_labels = test_generator.classes

# Get the class names from the test generator
class_names = list(test_generator.class_indices.keys())

# Get a batch of test images and their corresponding labels
images, _ = test_generator.next()

# Plot the images with their original and predicted labels
plt.figure(figsize=(10, 6))
for i in range(4):
    plt.subplot(2, 2, i + 1)
    plt.imshow(images[i])
    plt.title(f"True: {class_names[true_labels[i]]}\nPredicted: {class_names[predicted_labels[i]]}")
    plt.axis('off')
plt.tight_layout()
plt.show()


# Other Customized Models
These are other models that were used for the experiments

Model 3:

In [ ]:
import tensorflow as tf

model2 = tf.keras.models.Sequential([
    # Conv_1
    tf.keras.layers.Conv2D(96, (11, 11), strides=4, padding='valid', activation='relu', input_shape=(224, 224, 3)),
    # Pooling_1
    tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='valid'),
    # Batch Normalization_1
    tf.keras.layers.BatchNormalization(),

    # Conv_2
    tf.keras.layers.Conv2D(256, (5, 5), strides=1, padding='same', activation='relu'),
    # Pooling_2
    tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='valid'),
    # Batch Normalization_2
    tf.keras.layers.BatchNormalization(),

    # Conv_3
    tf.keras.layers.Conv2D(384, (3, 3), strides=1, padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    # Conv_4
    tf.keras.layers.Conv2D(384, (3, 3), strides=1, padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    # Conv_5
    tf.keras.layers.Conv2D(256, (3, 3), strides=1, padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    # Pooling_3
    tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='valid'),
    # Batch Normalization_3
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Flatten(),

    # Dense layer_1
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    # Batch Normalization_4
    tf.keras.layers.BatchNormalization(),

    # Dense layer_2
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    # Batch Normalization_5
    tf.keras.layers.BatchNormalization(),

    # Dense layer_3
    tf.keras.layers.Dense(20, activation='softmax')
])


In [ ]:
model2.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(learning_rate=1e-4),
              metrics=['acc'])

In [ ]:
# Train the model
history2 = model2.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.n//batch_s,
      epochs=100,
      validation_data=validation_generator,
      validation_steps=validation_generator.n//batch_s,
      verbose=2)

In [ ]:
model2.evaluate_generator(test_generator)


In [ ]:
save_model(model2, 'model2.h5')

In [ ]:
import matplotlib.pyplot as plt
acc = history2.history['acc']
val_acc = history2.history['val_acc']
loss = history2.history['loss']
val_loss = history2.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

Model 4:
Removing one convolution layer

In [ ]:
import tensorflow as tf

model3 = tf.keras.models.Sequential([
    # Conv_1
    tf.keras.layers.Conv2D(96, (11, 11), strides=4, padding='valid', activation='relu', input_shape=(224, 224, 3)),
    # Pooling_1
    tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='valid'),
    # Batch Normalization_1
    tf.keras.layers.BatchNormalization(),

    # Conv_2
    tf.keras.layers.Conv2D(256, (5, 5), strides=1, padding='same', activation='relu'),
    # Pooling_2
    tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='valid'),
    # Batch Normalization_2
    tf.keras.layers.BatchNormalization(),

    # Conv_3
    tf.keras.layers.Conv2D(384, (3, 3), strides=1, padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    # Conv_4
    tf.keras.layers.Conv2D(384, (3, 3), strides=1, padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    #Removed conv_5
    # Pooling_3
    tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='valid'),
    # Batch Normalization_3
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Flatten(),

    # Dense layer_1
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    # Batch Normalization_4
    tf.keras.layers.BatchNormalization(),

    # Dense layer_2
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    # Batch Normalization_5
    tf.keras.layers.BatchNormalization(),

    # Dense layer_3
    tf.keras.layers.Dense(20, activation='softmax')
])


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model3.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['acc'])

In [ ]:
# Train the model
history3 = model3.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.n//batch_s,
      epochs=100,
      validation_data=validation_generator,
      validation_steps=validation_generator.n//batch_s,
      verbose=2)

In [ ]:
model3.evaluate_generator(test_generator)

In [ ]:
import matplotlib.pyplot as plt
acc = history3.history['acc']
val_acc = history3.history['val_acc']
loss = history3.history['loss']
val_loss = history3.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()